# Category: Disposition

Disposition is the attempt to categorise KOI's into CANDIDATE or FALSE POSITIVE (with a score). 

This section also contains a list of boolean and string status flags (which could be one-hot-encoded = 126 fields)

In [1]:
# Python imports and settings
import numpy  as np
import pandas as pd
import re
from pydash import py_ as _
from src.dataset_koi import koi, koi_columns, koi_column_types
from src.utilities import onehot_encode_comments

# https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns-of-a-pandas-dataframe
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
pd.set_option('display.max_rows', 8)  # 8 is required for .describe()

%load_ext autoreload
%autoreload 2

### Load Dataset

In [2]:
dataset = koi['disposition']; 
dataset

,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_comment
kepoi_name,,,,,,,
K00752.01,CANDIDATE,1.000,0,0,0,0,NO_COMMENT
K00752.02,CANDIDATE,0.969,0,0,0,0,NO_COMMENT
K00753.01,CANDIDATE,0.000,0,0,0,0,DEEP_V_SHAPED
K00754.01,FALSE POSITIVE,0.000,0,1,0,0,MOD_ODDEVEN_DV---MOD_ODDEVEN_ALT---DEEP_V_SHAPED
...,...,...,...,...,...,...,...
K07986.01,CANDIDATE,0.497,0,0,0,0,NO_COMMENT
K07987.01,FALSE POSITIVE,0.021,0,0,1,0,CENT_RESOLVED_OFFSET
K07988.01,CANDIDATE,0.092,0,0,0,0,ALL_TRANS_CHASES---CENT_SATURATED
K07989.01,FALSE POSITIVE,0.000,0,0,1,1,CENT_RESOLVED_OFFSET---HALO_GHOST---EPHEM_MATCH


In [5]:
onehot_comments = onehot_encode_comments(dataset, 'koi_comment', '---')
onehot_dataset  = dataset.drop(columns=['koi_comment']).join(onehot_comments)
onehot_dataset

,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_comment_ALL_TRANS_CHASES,koi_comment_ALT_ROBO_ODD_EVEN_TEST_FAIL,koi_comment_ALT_SEC_SAME_DEPTH_AS_PRI_COULD_BE_TWICE_TRUE_PERIOD,koi_comment_ALT_SIG_PRI_MINUS_SIG_POS_TOO_LOW,koi_comment_ALT_SIG_PRI_MINUS_SIG_TER_TOO_LOW,koi_comment_ALT_SIG_PRI_OVER_FRED_TOO_LOW,koi_comment_CENTROID_SIGNIF_UNCERTAIN,koi_comment_CENT_CROWDED,koi_comment_CENT_FEW_DIFFS,koi_comment_CENT_FEW_MEAS,koi_comment_CENT_KIC_POS,koi_comment_CENT_NOFITS,koi_comment_CENT_RESOLVED_OFFSET,koi_comment_CENT_SATURATED,koi_comment_CENT_UNCERTAIN,koi_comment_CENT_UNRESOLVED_OFFSET,koi_comment_CLEAR_APO,koi_comment_CROWDED_DIFF,koi_comment_DEEP_V_SHAPED,koi_comment_DEPTH_ODDEVEN_ALT,koi_comment_DEPTH_ODDEVEN_DV,koi_comment_DV_SIG_PRI_MINUS_SIG_POS_TOO_LOW,koi_comment_DV_SIG_PRI_OVER_FRED_TOO_LOW,koi_comment_EPHEM_MATCH,koi_comment_EYEBALL,koi_comment_FIT_FAILED,koi_comment_HALO_GHOST,koi_comment_HAS_SEC_TCE,koi_comment_INCONSISTENT_TRANS,koi_comment_INDIV_TRANS_CHASES,koi_comment_INDIV_TRANS_CHASES_MARSHALL,koi_comment_INDIV_TRANS_CHASES_MARSHALL_SKYE,koi_comment_INDIV_TRANS_CHASES_MARSHALL_ZUMA,koi_comment_INDIV_TRANS_CHASES_SKYE,koi_comment_INDIV_TRANS_MARSHALL,koi_comment_INDIV_TRANS_MARSHALL_SKYE,koi_comment_INDIV_TRANS_MARSHALL_ZUMA,koi_comment_INDIV_TRANS_RUBBLE,koi_comment_INDIV_TRANS_RUBBLE_MARSHALL_SKYE,koi_comment_INDIV_TRANS_RUBBLE_SKYE,koi_comment_INDIV_TRANS_RUBBLE_SKYE_ZUMA,koi_comment_INDIV_TRANS_RUBBLE_SKYE_ZUMA_TRACKER,koi_comment_INDIV_TRANS_SKYE,koi_comment_INDIV_TRANS_SKYE_ZUMA,koi_comment_INDIV_TRANS_SKYE_ZUMA_TRACKER,koi_comment_INDIV_TRANS_ZUMA,koi_comment_INVERT_DIFF,koi_comment_IS_SEC_TCE,koi_comment_KIC_OFFSET,koi_comment_LPP_ALT,koi_comment_LPP_ALT_TOO_HIGH,koi_comment_LPP_DV,koi_comment_LPP_DV_TOO_HIGH,koi_comment_MARSHALL_FAIL,koi_comment_MOD_NONUNIQ_ALT,koi_comment_MOD_NONUNIQ_DV,koi_comment_MOD_ODDEVEN_ALT,koi_comment_MOD_ODDEVEN_DV,koi_comment_MOD_POS_ALT,koi_comment_MOD_POS_DV,koi_comment_MOD_SEC_ALT,koi_comment_MOD_SEC_DV,koi_comment_MOD_TER_ALT,koi_comment_MOD_TER_DV,koi_comment_NO_COMMENT,koi_comment_OTHER_TCE_AT_SAME_PERIOD_DIFF_EPOCH,koi_comment_PARENT_IS_002305372-pri,koi_comment_PARENT_IS_002449084-pri,koi_comment_PARENT_IS_003352751-pri,koi_comment_PARENT_IS_003858884-01,koi_comment_PARENT_IS_004482641-01,koi_comment_PARENT_IS_005024292-01,koi_comment_PARENT_IS_005036538-01,koi_comment_PARENT_IS_005343976-pri,koi_comment_PARENT_IS_005471619-pri,koi_comment_PARENT_IS_005513861-pri,koi_comment_PARENT_IS_006367628-pri,koi_comment_PARENT_IS_007258889-pri,koi_comment_PARENT_IS_007598128-pri,koi_comment_PARENT_IS_008265951-pri,koi_comment_PARENT_IS_008380743-pri,koi_comment_PARENT_IS_009541127-pri,koi_comment_PARENT_IS_009777062-01,koi_comment_PARENT_IS_010485137-pri,koi_comment_PARENT_IS_010858720-pri,koi_comment_PARENT_IS_012004679-pri,koi_comment_PARENT_IS_3597.01,koi_comment_PARENT_IS_3895.01,koi_comment_PARENT_IS_4673.01,koi_comment_PARENT_IS_489.01,koi_comment_PARENT_IS_5335.01,koi_comment_PARENT_IS_970.01,koi_comment_PARENT_IS_FL-Lyr-pri,koi_comment_PARENT_IS_RR-Lyr-pri,koi_comment_PARENT_IS_UZ-Lyr-pri,koi_comment_PARENT_IS_V2277-Cyg-pri,koi_comment_PARENT_IS_V380-Cyg-pri,koi_comment_PARENT_IS_V380-Cyg-sec,koi_comment_PARENT_IS_V850-Cyg-pri,koi_comment_PERIOD_ALIAS_ALT,koi_comment_PERIOD_ALIAS_DV,koi_comment_PERIOD_ALIAS_IN_ALT_DATA_SEEN_AT_3:1,koi_comment_PERIOD_ALIAS_IN_DV_DATA_SEEN_AT_3:1,koi_comment_PLANET_IN_STAR,koi_comment_PLANET_OCCULT_ALT,koi_comment_PLANET_OCCULT_DV,koi_comment_PLANET_PERIOD_IS_HALF_ALT,koi_comment_PLANET_PERIOD_IS_HALF_DV,koi_comment_RESIDUAL_TCE,koi_comment_RESID_OF_PREV_TCE,koi_comment_SAME_NTL_PERIOD,koi_comment_SAME_P_AS_PREV_NTL_TCE,koi_comment_SATURATED,koi_comment_SEASONAL_DEPTH_ALT,koi_comment_SEASONAL_DEPTH_DIFFS_IN_ALT,koi_comment_SEASONAL_DEPTH_DV,koi_comment_SIGNIF_OFFSET,koi_comment_SIG_SEC_IN_ALT_MODEL_SHIFT,koi_comment_SIG_SEC_IN_DV_MODEL_SHIFT,koi_comment_SWEET_EB,koi_comment_SWEET_NTL,koi_com